In [ ]:
# スクリーニング手法の探索
import datetime
from pathlib import Path

import polars as pl
import numpy as np
import matplotlib.pyplot as plt

import stock
from stock.trend_template import check_growing

In [ ]:
code_list = stock.kabutan.get_code_list()
current_date = datetime.date(year=2022, month=6, day=8)
window_size = 2
growing_rate = 0.15
duration = 2

In [ ]:
def get_simulation_results(
    watch_list: list[str], target_date: datetime.date = datetime.date.today()
):
    results = []
    for idx, code in enumerate(watch_list):
        stop_condition = stock.simulation.OnielStopCondition()
        res = stock.simulation.run(code, target_date, stop_condition)
        if res.buying_price > 0:
            results.append(
                {"code": code, "duration": res.duration, "profit": round(res.profit * 100)}
            )
    return results

In [ ]:
# 4半期の売上高、営業利益が前年同期比15%以上増加している銘柄
watch_list = []
for code in code_list:
    df = stock.kabutan.read_financial_csv(stock.DATA_DIR / f"financial/{code}.csv").filter(
        (pl.col("annoounce_date") <= current_date) & (pl.col("duration") == 3)
    ).sort(pl.col("annoounce_date"))
    if len(df) == 0:
        continue

    if current_date - df["annoounce_date"][-1] > datetime.timedelta(days=30):
        continue
    
    if (check_growing(df, "total_revenue", growing_rate, min_duration=duration,  current_date=current_date, num_average=window_size) and 
        check_growing(df, "operating_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size) and 
        check_growing(df, "net_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size)):
        watch_list.append(code)


In [ ]:
# nikkei, topixと比べてのパフォーマンスチェック
watching = []
not_watching = []
for code in code_list:
    df = stock.kabutan.read_data_csv(stock.DATA_DIR / f"daily/{code}.csv").filter(
        pl.col("date").is_between(current_date, upper_bound=current_date + datetime.timedelta(days=30), closed="right")
    )
    if len(df) < 10:
        continue
    rate = df["close"][-1] / df["open"][0]
    if code in watch_list:
        watching.append(rate)
    else:
        not_watching.append(rate)

In [ ]:
df = stock.kabutan.read_data_csv(stock.DATA_DIR / f"daily/0010.csv").filter(
        pl.col("date").is_between(current_date, upper_bound=current_date + datetime.timedelta(days=30), closed="right")
    )
rate = df["close"][-1] / df["open"][0]

In [ ]:
results = get_simulation_results(code_list, target_date=current_date)

In [ ]:
success_list = [res["code"] for res in results if res["profit"] >= 20]
growing_list = []

for code in success_list:
    df = stock.kabutan.read_financial_csv(stock.DATA_DIR / f"financial/{code}.csv").filter(
        (pl.col("annoounce_date") <= current_date) & (pl.col("duration") == 3)
    ).sort(pl.col("annoounce_date"))
    if len(df) == 0:
        continue

    if current_date - df["annoounce_date"][-1] > datetime.timedelta(days=30):
        continue
    
    if (check_growing(df, "total_revenue", growing_rate, min_duration=duration,  current_date=current_date, num_average=window_size) and 
        check_growing(df, "operating_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size) and 
        check_growing(df, "net_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size)):
        growing_list.append(code)

In [ ]:
# relative strengthの検証
current_date = datetime.date(year=2022, month=6, day=8)

# relative strengthの高い銘柄を取得
target_key = "rs"
target_code_list = []
target_rs = {}
rs_list = []
for code in code_list:
    df = stock.kabutan.read_data_csv(csv_path=stock.DATA_DIR / f"daily/{code}.csv", end_date=current_date)
    if len(df) > 0 and df[target_key][-1] > 0:
        rs_list.append([df[target_key][-1], df["rs_topix"][-1], df["rs"][-1]])
        target_code_list.append(code)

ind = np.argsort(rs_list, axis=0)
start_pos = int(len(rs_list) * 0.9)
end_pos = int(len(rs_list) * 0.95)
target_indices = ind[start_pos:end_pos, 0]
target_code = np.array(target_code_list)[target_indices]
target_rs[target_key] = np.array(rs_list)[target_indices, 0]

In [ ]:
# simulation結果を表示
sim_res = get_simulation_results(target_code, target_date=current_date)
grid = np.array([[rs, res["profit"]] for rs, res in zip(target_rs[target_key], sim_res)])
num_success = (grid[:, 1] >= 20).sum()
print("{} / {} ({:.2f} %)".format(num_success, len(grid), num_success / len(grid) * 100))

In [ ]:
# テクニカルでスクリーニング
watch_list = []
for code in target_code:
    df = stock.kabutan.read_data_csv(stock.DATA_DIR / f"daily/{code}.csv", end_date=current_date)
    # relative strengthが上昇している
    max_rs = df.filter(pl.col("date") >= current_date - datetime.timedelta(days=10))["rs"].max()
    if max_rs * 0.95 > df["rs"][-1]:
        continue

    # 新高値付近にある
    highest = df.filter(pl.col("date") >= current_date - datetime.timedelta(days=90))["high"].max()
    if highest * 0.80 > df["close"][-1]:
        continue

    watch_list.append(code)

In [ ]:
# simulation結果を表示
sim_res = get_simulation_results(watch_list, target_date=current_date)
grid = np.array([res["profit"] for res in sim_res])
num_success = (grid >= 20).sum()
print("{} / {} ({:.2f} %)".format(num_success, len(grid), num_success / len(grid) * 100))

In [ ]:
code_list = stock.kabutan.get_code_list()
current_date = datetime.date(year=2022, month=6, day=8)
window_size = 2
growing_rate = 0.1
duration = 2

In [ ]:
target_code = watch_list

In [ ]:
# fundamentalsでスクリーニング
watch_list = []
for code in target_code:
    df = stock.kabutan.read_financial_csv(stock.DATA_DIR / f"financial/{code}.csv").filter(
        (pl.col("annoounce_date") <= current_date) & (pl.col("duration") == 3)
    ).sort(pl.col("annoounce_date"))
    # if len(df) == 0:
    #     watch_list.append(code)
    #     continue

    # if current_date - df["annoounce_date"][-1] > datetime.timedelta(days=30):
    #     watch_list.append(code)
    #     continue
    
    if not (check_growing(df, "total_revenue", growing_rate, min_duration=duration,  current_date=current_date, num_average=window_size) or
        check_growing(df, "operating_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size) or
        check_growing(df, "net_income", growing_rate, min_duration=duration, current_date=current_date, num_average=window_size)):
        if stock.kabutan.data.get_market_capitalization(code) > 50:
            watch_list.append(code)    

In [ ]:
# simulation結果を表示
sim_res = get_simulation_results(watch_list, target_date=current_date)
grid = np.array([res["profit"] for res in sim_res])
num_success = (grid >= 20).sum()
print("{} / {} ({:.2f} %)".format(num_success, len(grid), num_success / len(grid) * 100))

In [ ]:
watch_list

In [ ]:
window_size = 2
growing_rate = 0.15
duration = 2

not (check_growing(
    df,
    "total_revenue",
    growing_rate,
    min_duration=duration,
    current_date=current_date,
    num_average=window_size,
) or 
check_growing(
    df,
    "operating_income",
    growing_rate,
    min_duration=duration,
    current_date=current_date,
    num_average=window_size,
) or
check_growing(
    df,
    "net_income",
    growing_rate,
    min_duration=duration,
    current_date=current_date,
    num_average=window_size,
)
)
